# Traffic Recorder Application Testing Code

This notebook is designed to communicate with the Traffic Recorder Application server and perform a series of tests. It is necessary to restart the server for each test to ensure that the test did not fail because of the state the server was in due to the previous tests. Each time the server restarts that ensures it begins with a database in a known state.

In [1]:
# The modules needed
import random
import requests
import shutil
import json
import subprocess
import sqlite3

import os
import socket

In [2]:
# The core code

def request2server_get(url,cookies):
    """Send a get request to the server"""
    ucookie = cookies[0]
    mcookie = cookies[1]
    try:
        r = requests.get(url,cookies=dict(u_cookie=cookies[0],m_cookie=cookies[1]),timeout=30)
        for c in r.cookies:
            if (c.name == 'u_cookie'):
                ucookie = c.value
            if (c.name == 'm_cookie'):
                mcookie = c.value
        act=json.loads(r.text)
        return [[ucookie,mcookie],act]
    except :
        print("Invalid login")
        return [[ucookie,mcookie],[]]

def request2server_post(url,cookies,content):
    """Send a post request to the server"""
    ucookie = cookies[0]
    mcookie = cookies[1]
    try:
        r = requests.post(url,cookies=dict(u_cookie=cookies[0],m_cookie=cookies[1]),json=content, timeout=30)
        for c in r.cookies:
            if (c.name == 'u_cookie'):
                ucookie = c.value
            if (c.name == 'm_cookie'):
                mcookie = c.value
        act=json.loads(r.text)
        return [[ucookie,mcookie],act]
    except :
        print("Invalid login")
        return [[ucookie,mcookie],[]]

def do_login(cookies,user,pasw):
    """Send a login command"""
    global server_port
    content = {"command":"login","username":user,"password":pasw}
    return request2server_post("http://localhost:"+server_port+"/action?command=login",cookies, content)

def do_logout(cookies):
    """Send a logout command"""
    global server_port
    content = {"command":"logout"}
    return request2server_post("http://localhost:"+server_port+"/action?command=logout",cookies, content)


def do_get_locations(cookies):
    """Send an add_activity command"""
    global server_port
    content = {"command":"location"}
    return request2server_post("http://localhost:"+server_port+"/action?command=location",cookies,content)

def do_get_summary(cookies, location):
    """Send an add_activity command"""
    global server_port
    content = {"command":"summary", "location":location}
    return request2server_post("http://localhost:"+server_port+"/action?command=summary",cookies,content)

def do_add(cookies,location,vtype,occupancy):
    """Send an add_activity command"""
    global server_port
    content = {"command":"add", "location":location,"type":vtype, "occupancy":occupancy}
    return request2server_post("http://localhost:"+server_port+"/action?command=add",cookies,content)

def do_undo(cookies,location,vtype,occupancy):
    """Send an add_activity command"""
    global server_port
    content = {"command":"undo", "location":location,"type":vtype, "occupancy":occupancy}
    return request2server_post("http://localhost:"+server_port+"/action?command=undo",cookies,content)



In [3]:
def do_login_incomplete(cookies,pasw):
    """Send an incomplete login command"""
    global server_port
    content = {"command":"login","password":pasw}
    return request2server_post("http://localhost:"+server_port+"/action?command=login",cookies, content)



In [4]:
def find_redirect(act):
    """Check for a redirect response. Return the where target if found or None otherwise."""
    if act == None:
        return None
    try:
        for a in act:
            if(a['type']=='redirect'):
                return a['where']
    except:
        return None

def find_message_zero(act):
    """Check if there is a message with code 0"""
    if act == None:
        return False
    try:
        for a in act:
            if(a['type']=='message'):
                code = a['code']
                if code == 0:
                    return True
    except:
        return False
    
def find_message_100(act):
    """Check if there is a message with code in the range 100-199"""
    if act == None:
        return False
    try:
        for a in act:
            if(a['type']=='message'):
                code = a['code']
                if code >= 100 and code <= 199:
                    return True
        return False
    except:
        return False
    
def find_message_200(act):
    """Check if there is a message with code in the range 200-299"""
    if act == None:
        return False
    try:
        for a in act:
            if(a['type']=='message'):
                code = a['code']
                if code >= 200 and code <= 299:
                    return True
        return False
    except:
        return False

def checked_login(test,cookies,user,pasw):
    """Send a login command and check it's good."""
    [cookies,act] = do_login(cookies, user,pasw)
    where = find_redirect(act)
    if(where == None):
        print("Test "+str(test)+" Failed - Expected redirect during login.")
        return ['',act,False]
    if (where != '/index.html'):
        print("Test "+str(test)+" Failed - Expected /index.html got {"+where+"}")
        return [cookies,act,False]
    return [cookies,act,True]

def checked_logout(test,cookies):
    """Send a logout command and check it's good."""
    [cookies,act] = do_logout(cookies)     
    where = find_redirect(act)
    if(where == None):
        print("Test "+str(test)+" Failed - Expected redirect during logout.")
        return [cookies,act,False]
    if (where != '/logout.html'):
        print("Test "+str(test)+" Failed - Expected /logout.html got {"+where+"}")
        return [cookies,act,False] 
    return [cookies,act,True]

In [5]:
# Some useful functions to access the database

def do_database_execute(op):
  """Execute an SQL command that is not expected to return any rows."""
  print(op)
  try:
    db = sqlite3.connect('database.db')
    cursor = db.cursor()
    cursor.execute(op)
    db.commit()
  except Exception as e:
    db.rollback()
  finally:
    db.close()

def do_database_fetchone(op):
  """Execute an SQL command that returns at most a single row."""
  print(op)
  try:
    db = sqlite3.connect('database.db')
    cursor = db.cursor()
    cursor.execute(op)
    result = cursor.fetchone()
    print(result)
    db.close()
    return result
  except Exception as e:
    print(e)
    return None

def do_database_fetchall(op):
  """Execute an SQL command that can return any number of rows, including none."""
  print(op)
  try:
    db = sqlite3.connect('database.db')
    cursor = db.cursor()
    cursor.execute(op)
    result = cursor.fetchall()
    print(result)
    db.close()
    return result
  except Exception as e:
    print(e)
    return None


In [6]:
# Test 1 - Simple login
def test1():
    """Check that login and logout work for a good user."""
    try:
        cookies = ['','']
        [cookies,act,flag] = checked_login(1,cookies,"test1","password1")
        if flag!=True:
            print("Test 1 Failed, bad login.")
            return 0

        [cookies,act,flag] = checked_logout(1,cookies)        
        if flag!=True:
            print("Test 1 Failed, bad logout.")
            return 0

        print("Test 1 Passed")
        return 1
    except:
        print("Test 1 Failed - Exception Caused.")
        return 0


In [7]:
# Test 2 - Login with bad password
def test2():
    """Check that login reports bad parameters, in this case a bad password."""
    try:
        cookies = ['','']
        [cookies,act] = do_login(cookies,"test1","notpassword")
        
        where = find_redirect(act)
        if (where != None):
            print("Test 2 Failed - Unexpected redirect.")
            return 0
        
        if find_message_200(act):
            print("Test 2 Failed - Unexpected missing parameter message code found.")
            return 0
        
        if find_message_100(act) == False:
            print("Test 2 Failed - Invalid parameter message not found.")
            return 0

        print("Test 2 Passed")
        return 1
    except:
        print("Test 2 Failed - Exception Caused.")
        return 0

In [8]:
# Test 3 - Login with missing username
def test3():
    """Check that login rejects attempts with missing parameters, in this case a missing username"""
    try:
        cookies = ['','']
        [cookies,act] = do_login_incomplete(cookies,"passwordX")
        
        where = find_redirect(act)
        if (where != None):
            print("Test 3 Failed - Unexpected redirect.")
            return 0
        
        if find_message_100(act):
            print("Test 3 Failed - Unexpected invalid parameter message code found.")
            return 0
        
        if find_message_zero(act):
            print("Test 3 Failed - Unexpected all good message found.")
            return 0
        
        if find_message_200(act) == False:
            print("Test 3 Failed - Missing parameter message not found.")
            return 0
        print("Test 3 Passed")
        return 1
    except:
        print("Test 3 Failed - Exception Caused.")
        return 0        

In [9]:
# Test 4 - Get the list of locations and it's a new session so zero total as well
def test4():
    """Check that we get the response we expect."""
    try:
        cookies = ['','']
        [cookies,act,flag] = checked_login(4,cookies,"test1","password1")

        if flag!=True:
            return 0
        
        [cookies,act] = do_get_locations(cookies)
      
        if len(act) == 0:
            print("Test 6 Failed - Expected some responses.")
            return 0
        
        lcount = 0
        tcount = 0
        
        try:
            for a in act:
                if a['type'] == 'location':
                    lcount += 1
                elif a['type'] == 'total':
                    tcount += 1
                    if a['total'] != 0:
                        print("Test 4 Failed - Total value is not zero.")
                        return 0
                elif a['type'] == 'redirect':
                        print("Test 5 Failed - Unexpected redirect.")
                        return 0
                elif a['type'] == 'message':
                    print("Test 4 Failed - Did not expect a message.")
                    return 0
                              
            if lcount != 4 and tcount != 1:
                print(f"Test 4 Failed - Expected 4 location and one total. {lcount} {tcount}")
                return 0
                        
        except:
            print("Test 4 Failed - Responses missing elements.")
            return 0
        
        print("Test 4 Passed")
        return 1
    except:
        print("Test 4 Failed - Exception Caused.")
        return 0

In [10]:
# Test 5 - Add a new record successfully
def test5():
    """Check that we get the response we expect."""
    try:
        cookies = ['','']
        [cookies,act,flag] = checked_login(5,cookies,"test1","password1")

        if flag!=True:
            return 0
        
        [cookies,act] = do_add(cookies,2,1,3) # location, type, occupancy
      
        if len(act) == 0:
            print("Test 6 Failed - Expected  responses.")
            return 0
        
        try:
            for a in act:
                if a['type'] == 'total':
                    if a['total'] != 1:
                        print("Test 5 Failed - Total value is not 1.")
                        return 0
                elif a['type'] == 'redirect':
                        print("Test 5 Failed - Unexpected redirect.")
                        return 0 
        except:
            print("Test 5 Failed - Responses missing elements.")
            return 0
        
                
        if find_message_100(act):
            print("Test 3 Failed - Unexpected invalid parameter message code found.")
            return 0
        
        if find_message_200(act):
            print("Test 3 Failed - Missing parameter message not found.")
            return 0
        
        if not find_message_zero(act):
            print("Test 3 Failed - Good message not found.")
            return 0
        

        
        print("Test 5 Passed")
        return 1
    except:
        print("Test 5 Failed - Exception Caused.")
        return 0

In [11]:
# Test 5b - After server ended, check database.
def test5b():
    print("Test 5 DB Check, After server ended, check database.")
    res = do_database_fetchall("SELECT * FROM TRAFFIC")
    if len(res) == 0:
        print("Test 5 DB Check, No traffic record found.")
        return 0
    if len(res) > 1:
        print("Test 5 DB Check, Too many traffic records found")
        return 0
    if res[0][3] != 1:
        print("Test 5 DB Check, Unexpected type")
        return 0
    if res[0][4] != 3:
        print("Test 5 DB Check, Unexpected occupancy")
        return 0
    if res[0][5] != 2:
        print("Test 5 DB Check, Unexpected locationid")
        return 0
    if res[0][6] != 1:
        print("Test 5 DB Check, Unexpected mode")
        return 0
    return 1

In [12]:
# Test 6 - Check existing summary
def test6():
    """Check that we get the response we expect."""
    try:
        cookies = ['1','96878932']
        
        [cookies,act] = do_get_summary(cookies,1)
      
        if len(act) == 0:
            print("Test 6 Failed - Expected  responses.")
            return 0
        
        try:
            for a in act:
                if a['type'] == 'total':
                    if a['total'] != 10:
                        print("Test 6 Failed - Total value is not 10.")
                        return 0
                elif a['type'] == 'redirect':
                        print("Test 6 Failed - Unexpected redirect.")
                        return 0
                elif a['type'] == 'vcount':
                        if a['vtype']==1:
                            if a['count'] != 2:
                                print("Test 6 Failed - Wrong summary count value")
                        elif a['vtype']==2:
                            if a['count'] != 1:
                                print("Test 6 Failed - Wrong summary count value")
                        else:
                            if a['count'] != 0:
                                print("Test 6 Failed - Unexpected non-zero count of vehicle type.")
                                return 0 
        except:
            print("Test 6 Failed - Responses missing elements.")
            return 0
        
                
        if find_message_100(act):
            print("Test 6 Failed - Unexpected invalid parameter message code found.")
            return 0
        
        if find_message_200(act):
            print("Test 6 Failed - Missing parameter message not found.")
            return 0
        
        if not find_message_zero(act):
            print("Test 6 Failed - Good message not found.")
            return 0
            
        print("Test 6 Passed")
        return 1
    except:
        print("Test 6 Failed - Exception Caused.")
        return 0

In [13]:
# The lists of tests, you can add your own here.
# test description, test function, database to use for test
tests = [('Test 1 Simple login test.', test1, 'db/example1.db', None,1),
         ('Test 2 Check bad password rejected.', test2, 'db/example1.db', None,1),
         ('Test 3 Check missing username on login is rejected', test3, 'db/example1.db', None,1),
         ('Test 4 Check location list is good.', test4, 'db/example1.db', None,1),
         ('Test 5 Add a good record', test5, 'db/example1.db', test5b,2),
         ('Test 6 Get summary of active session', test6, 'db/example2.db', None,1)
]

def test_suite(file_to_run):
    """Iterate over all the tests and run them with a fresh server and copy of the specified database"""
    global server_port # we cycle through ports if a program fails and occupies the port

    mark_total = 0 # marks for passsed tests
    mark_max = 0 # maximum possible marks
    
    print("About to run test suite")
     
    # loop over the tests
    for tnumber, test in enumerate(tests, start = 1):
        
        #make sure we are using a free port
        port = int(server_port)
        busy = 1
        while busy == 1:
            sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
            sock.settimeout(2)
            result = sock.connect_ex(('localhost',port))
            if result == 0:
                print('Port',port,'busy')
                port +=1
            else:
                print('Using port',port)
                busy = 0
                
        server_port = str(port)
                
        # try the server test, if it generates an exception that is considered a fail.
        try:
            print("Server Test Starting,",test[0])
            mark_max += test[4]
            
            # copy the database to where the server expects it
            shutil.copy(test[2],'database.db') 
            
            #start the server
            sp = subprocess.Popen('C:\Program Files\Anaconda3\python.exe '+ file_to_run +' '+server_port)
            
            # run the test and record the mark.
            tmark = test[1]()
            mark_total += tmark

            # close down the server
            sp.terminate()
            
            print("Server Test Finished")
            
        except Exception as err:
            print(err)
            try:
                sp.terminate()
            except:
                pass
            print("Server Test Process Generated Exception")
            
        try:
            if (test[3] != None):
                print("Database Test Starting")
                
                tmark = test[3]()
                mark_total += tmark
                
                print("Database Test Finished")
                
        except Exception as err:
            print(err)
            print("Database Test Process Generated Exception")
                
            
        tnumber += 1
        
    print(f"Marks = {mark_total}/{mark_max} for regression tests.")


In [15]:
server_port=8081
file_to_run = 'server.py'
test_suite(file_to_run)

About to run test suite
Using port 8081
Server Test Starting, Test 1 Simple login test.
Test 1 Passed
Server Test Finished
Using port 8081
Server Test Starting, Test 2 Check bad password rejected.
Test 2 Passed
Server Test Finished
Using port 8081
Server Test Starting, Test 3 Check missing username on login is rejected
Test 3 Passed
Server Test Finished
Port 8081 busy
Using port 8082
Server Test Starting, Test 4 Check location list is good.
Test 4 Passed
Server Test Finished
Port 8082 busy
Using port 8083
Server Test Starting, Test 5 Add a good record
Test 5 Passed
Server Test Finished
Database Test Starting
Test 5 DB Check, After server ended, check database.
SELECT * FROM TRAFFIC
[(1, 1, 1729842920, 1, 3, 2, 1)]
Database Test Finished
Using port 8083
Server Test Starting, Test 6 Get summary of active session
Test 6 Passed
Server Test Finished
Marks = 7/7 for regression tests.
